In [1]:
import pandas as pd
import numpy as np
import anndata
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import warnings
import pathlib

warnings.filterwarnings('ignore')

In [2]:
from matplotlib import rc
labelsize = 6
linewidth = 0.6
rc('lines', linewidth=linewidth)
rc('axes', labelsize=labelsize, linewidth=linewidth)
rc('xtick', labelsize=labelsize)
rc('ytick', labelsize=labelsize)
rc('xtick.major', width=linewidth)
rc('ytick.major', width=linewidth)
rc('xtick.minor', width=linewidth - 0.2)
rc('ytick.minor', width=linewidth - 0.2)

In [3]:
chrom = 'chr2'
genes = ['Lhx6']
slop = 250000
n_pc = 10
resolution = 1

In [4]:
# Parameters
chrom = "chr8"
genes = ["ENSMUSG00000025812.17", "ENSMUSG00000025810.9", "ENSMUSG00000079157.4", "ENSMUSG00000063626.6", "ENSMUSG00000010936.11", "ENSMUSG00000031618.13", "ENSMUSG00000001911.16", "ENSMUSG00000041949.6", "ENSMUSG00000037492.16", "ENSMUSG00000046844.6", "ENSMUSG00000043668.13", "ENSMUSG00000031822.20", "ENSMUSG00000031642.9", "ENSMUSG00000060924.15", "ENSMUSG00000045636.16", "ENSMUSG00000039057.11", "ENSMUSG00000031930.11", "ENSMUSG00000036306.13", "ENSMUSG00000036356.15", "ENSMUSG00000041308.6", "ENSMUSG00000031565.18", "ENSMUSG00000062991.8", "ENSMUSG00000001995.9", "ENSMUSG00000031709.15", "ENSMUSG00000031561.16", "ENSMUSG00000036469.16", "ENSMUSG00000057060.8", "ENSMUSG00000089704.9", "ENSMUSG00000071064.14", "ENSMUSG00000031523.16", "ENSMUSG00000031841.19", "ENSMUSG00000047747.10", "ENSMUSG00000037148.8", "ENSMUSG00000031681.16", "ENSMUSG00000036510.17", "ENSMUSG00000031608.13", "ENSMUSG00000031453.16", "ENSMUSG00000000759.14", "ENSMUSG00000053626.5", "ENSMUSG00000031772.17", "ENSMUSG00000038291.16", "ENSMUSG00000031824.15", "ENSMUSG00000031673.6", "ENSMUSG00000070056.6", "ENSMUSG00000031626.17", "ENSMUSG00000001300.16", "ENSMUSG00000045333.15", "ENSMUSG00000086741.1", "ENSMUSG00000074203.4", "ENSMUSG00000036913.15", "ENSMUSG00000031954.6", "ENSMUSG00000031441.15", "ENSMUSG00000055932.15", "ENSMUSG00000054162.15", "ENSMUSG00000097393.8", "ENSMUSG00000031502.11", "ENSMUSG00000030465.19", "ENSMUSG00000031503.13", "ENSMUSG00000110286.1", "ENSMUSG00000052479.5", "ENSMUSG00000037940.17", "ENSMUSG00000004637.15", "ENSMUSG00000110369.1", "ENSMUSG00000031683.16", "ENSMUSG00000031837.14", "ENSMUSG00000031563.8", "ENSMUSG00000039842.15", "ENSMUSG00000110523.1", "ENSMUSG00000031577.10", "ENSMUSG00000036180.15", "ENSMUSG00000031529.5", "ENSMUSG00000033192.5", "ENSMUSG00000031559.15", "ENSMUSG00000031906.9", "ENSMUSG00000110485.1", "ENSMUSG00000031465.6", "ENSMUSG00000090116.1", "ENSMUSG00000109994.1", "ENSMUSG00000031983.7", "ENSMUSG00000096914.3", "ENSMUSG00000015568.16", "ENSMUSG00000055435.6", "ENSMUSG00000037101.16", "ENSMUSG00000043051.17", "ENSMUSG00000034361.10", "ENSMUSG00000038872.10", "ENSMUSG00000058056.16", "ENSMUSG00000031684.11", "ENSMUSG00000110246.1", "ENSMUSG00000038143.16", "ENSMUSG00000031732.12", "ENSMUSG00000110455.1", "ENSMUSG00000085440.1", "ENSMUSG00000100335.2", "ENSMUSG00000086451.8", "ENSMUSG00000031785.16", "ENSMUSG00000037852.8", "ENSMUSG00000031826.20", "ENSMUSG00000034656.18", "ENSMUSG00000065954.11", "ENSMUSG00000084954.2", "ENSMUSG00000053693.16", "ENSMUSG00000031666.15", "ENSMUSG00000096188.1", "ENSMUSG00000097248.9", "ENSMUSG00000033931.9", "ENSMUSG00000047935.17", "ENSMUSG00000108912.1", "ENSMUSG00000031605.8", "ENSMUSG00000031883.13", "ENSMUSG00000031481.8", "ENSMUSG00000036872.16", "ENSMUSG00000039620.17", "ENSMUSG00000056608.14", "ENSMUSG00000110336.1", "ENSMUSG00000033998.9", "ENSMUSG00000053399.9", "ENSMUSG00000031578.5", "ENSMUSG00000050271.12", "ENSMUSG00000014786.8", "ENSMUSG00000033763.14", "ENSMUSG00000031758.10", "ENSMUSG00000037656.9", "ENSMUSG00000109372.1", "ENSMUSG00000031652.11", "ENSMUSG00000039960.5", "ENSMUSG00000070047.14", "ENSMUSG00000037921.15", "ENSMUSG00000109961.1", "ENSMUSG00000031444.16", "ENSMUSG00000031451.6", "ENSMUSG00000039375.16", "ENSMUSG00000109438.1", "ENSMUSG00000109727.1", "ENSMUSG00000110329.1", "ENSMUSG00000110684.1", "ENSMUSG00000036902.11", "ENSMUSG00000109701.1", "ENSMUSG00000042353.9", "ENSMUSG00000015016.8", "ENSMUSG00000014633.15", "ENSMUSG00000089820.1", "ENSMUSG00000036712.15", "ENSMUSG00000051978.9", "ENSMUSG00000031647.10", "ENSMUSG00000031530.6", "ENSMUSG00000033282.14", "ENSMUSG00000102046.1", "ENSMUSG00000041438.8", "ENSMUSG00000031727.8", "ENSMUSG00000025044.16", "ENSMUSG00000032939.14", "ENSMUSG00000031827.13", "ENSMUSG00000003812.13", "ENSMUSG00000109127.1", "ENSMUSG00000109966.1", "ENSMUSG00000034390.16", "ENSMUSG00000031627.9", "ENSMUSG00000109682.1", "ENSMUSG00000025316.16", "ENSMUSG00000034796.14", "ENSMUSG00000038257.9", "ENSMUSG00000031788.14", "ENSMUSG00000031955.10", "ENSMUSG00000109870.1", "ENSMUSG00000039199.6", "ENSMUSG00000031482.9", "ENSMUSG00000006567.11", "ENSMUSG00000045248.6", "ENSMUSG00000031774.8", "ENSMUSG00000031821.11", "ENSMUSG00000110217.1", "ENSMUSG00000039431.16", "ENSMUSG00000006362.16", "ENSMUSG00000110554.1", "ENSMUSG00000038894.7", "ENSMUSG00000031910.14", "ENSMUSG00000037437.18", "ENSMUSG00000031987.6", "ENSMUSG00000109699.1", "ENSMUSG00000039661.14", "ENSMUSG00000031959.15", "ENSMUSG00000084848.1", "ENSMUSG00000109995.1", "ENSMUSG00000031590.8", "ENSMUSG00000031616.7", "ENSMUSG00000109827.1", "ENSMUSG00000039470.15", "ENSMUSG00000031595.9", "ENSMUSG00000008206.16", "ENSMUSG00000036330.12", "ENSMUSG00000097026.1", "ENSMUSG00000037022.3", "ENSMUSG00000089819.1", "ENSMUSG00000048148.18", "ENSMUSG00000034416.18", "ENSMUSG00000074113.4", "ENSMUSG00000086134.1", "ENSMUSG00000089745.1", "ENSMUSG00000038173.15", "ENSMUSG00000085573.1", "ENSMUSG00000050052.12", "ENSMUSG00000074491.9", "ENSMUSG00000031885.14", "ENSMUSG00000035151.12", "ENSMUSG00000031789.17", "ENSMUSG00000031620.16", "ENSMUSG00000046152.16", "ENSMUSG00000110559.1", "ENSMUSG00000037363.9", "ENSMUSG00000109162.1", "ENSMUSG00000110629.1", "ENSMUSG00000031750.15", "ENSMUSG00000039539.13", "ENSMUSG00000031508.14", "ENSMUSG00000059854.8", "ENSMUSG00000031872.14", "ENSMUSG00000031835.16", "ENSMUSG00000109371.1", "ENSMUSG00000109741.1", "ENSMUSG00000110719.1", "ENSMUSG00000031714.9", "ENSMUSG00000031576.16", "ENSMUSG00000106471.1", "ENSMUSG00000109819.1", "ENSMUSG00000031568.16", "ENSMUSG00000097992.1", "ENSMUSG00000031586.17", "ENSMUSG00000003316.14", "ENSMUSG00000031748.16", "ENSMUSG00000018796.13", "ENSMUSG00000070084.3", "ENSMUSG00000031730.18", "ENSMUSG00000054320.16", "ENSMUSG00000109838.1", "ENSMUSG00000006276.10", "ENSMUSG00000046691.14", "ENSMUSG00000117748.1", "ENSMUSG00000097520.2", "ENSMUSG00000110264.1", "ENSMUSG00000110363.1", "ENSMUSG00000109907.1", "ENSMUSG00000084960.1", "ENSMUSG00000074235.4", "ENSMUSG00000071176.10", "ENSMUSG00000031767.13", "ENSMUSG00000109689.1", "ENSMUSG00000031538.6", "ENSMUSG00000110697.1", "ENSMUSG00000040010.10", "ENSMUSG00000031512.15", "ENSMUSG00000022322.8", "ENSMUSG00000110063.1", "ENSMUSG00000109139.1", "ENSMUSG00000089956.1", "ENSMUSG00000048827.12", "ENSMUSG00000095279.2", "ENSMUSG00000110467.1", "ENSMUSG00000110592.1", "ENSMUSG00000110517.1", "ENSMUSG00000031749.12", "ENSMUSG00000002346.16", "ENSMUSG00000097695.1", "ENSMUSG00000099419.1", "ENSMUSG00000031949.17", "ENSMUSG00000109688.1", "ENSMUSG00000031480.15", "ENSMUSG00000049577.15", "ENSMUSG00000109714.1", "ENSMUSG00000036564.17", "ENSMUSG00000051648.13", "ENSMUSG00000109568.1", "ENSMUSG00000110333.1", "ENSMUSG00000108918.1", "ENSMUSG00000034472.13", "ENSMUSG00000039633.6", "ENSMUSG00000085197.7", "ENSMUSG00000110428.1", "ENSMUSG00000110340.1", "ENSMUSG00000036879.15", "ENSMUSG00000086866.2", "ENSMUSG00000031584.16", "ENSMUSG00000031557.16", "ENSMUSG00000110534.1", "ENSMUSG00000055675.6", "ENSMUSG00000031511.15", "ENSMUSG00000109672.1", "ENSMUSG00000074497.5", "ENSMUSG00000031520.6", "ENSMUSG00000109529.1", "ENSMUSG00000014444.17", "ENSMUSG00000047866.18", "ENSMUSG00000050079.10", "ENSMUSG00000033596.5", "ENSMUSG00000043687.15", "ENSMUSG00000047388.10", "ENSMUSG00000110013.1", "ENSMUSG00000031555.15", "ENSMUSG00000031974.8", "ENSMUSG00000037738.11", "ENSMUSG00000109545.1", "ENSMUSG00000097000.2", "ENSMUSG00000031825.16", "ENSMUSG00000079070.10", "ENSMUSG00000110698.1", "ENSMUSG00000109095.1", "ENSMUSG00000031583.13", "ENSMUSG00000038225.15", "ENSMUSG00000000739.14", "ENSMUSG00000098684.1", "ENSMUSG00000104292.1", "ENSMUSG00000109286.1", "ENSMUSG00000110269.1", "ENSMUSG00000109769.1", "ENSMUSG00000039328.11", "ENSMUSG00000109825.1", "ENSMUSG00000109964.1", "ENSMUSG00000109040.1", "ENSMUSG00000110557.1", "ENSMUSG00000031665.7", "ENSMUSG00000031562.15", "ENSMUSG00000053886.4", "ENSMUSG00000109551.1", "ENSMUSG00000098322.1", "ENSMUSG00000050751.14", "ENSMUSG00000085150.1", "ENSMUSG00000031461.4", "ENSMUSG00000110260.1", "ENSMUSG00000097919.1", "ENSMUSG00000031823.14", "ENSMUSG00000071047.5", "ENSMUSG00000031527.7", "ENSMUSG00000031534.15", "ENSMUSG00000110412.1", "ENSMUSG00000034330.10", "ENSMUSG00000109436.1", "ENSMUSG00000031698.14", "ENSMUSG00000003575.13", "ENSMUSG00000013033.16", "ENSMUSG00000110456.1", "ENSMUSG00000062007.13", "ENSMUSG00000031621.9", "ENSMUSG00000101854.2", "ENSMUSG00000083285.2", "ENSMUSG00000057400.14", "ENSMUSG00000031851.14", "ENSMUSG00000031635.14", "ENSMUSG00000031790.8", "ENSMUSG00000110089.1", "ENSMUSG00000031517.8", "ENSMUSG00000031833.10", "ENSMUSG00000096938.7", "ENSMUSG00000097380.1", "ENSMUSG00000109583.1", "ENSMUSG00000060098.11", "ENSMUSG00000109473.1", "ENSMUSG00000109385.1", "ENSMUSG00000084780.7", "ENSMUSG00000031545.6", "ENSMUSG00000056018.14", "ENSMUSG00000069922.12", "ENSMUSG00000110599.1", "ENSMUSG00000109772.1", "ENSMUSG00000079058.3", "ENSMUSG00000086681.1", "ENSMUSG00000079011.9", "ENSMUSG00000110242.1", "ENSMUSG00000031492.13", "ENSMUSG00000031979.17", "ENSMUSG00000083586.2", "ENSMUSG00000085678.1", "ENSMUSG00000110160.1", "ENSMUSG00000101899.1", "ENSMUSG00000109260.1", "ENSMUSG00000035057.7", "ENSMUSG00000014782.16", "ENSMUSG00000074300.4", "ENSMUSG00000031445.5", "ENSMUSG00000110616.1", "ENSMUSG00000110084.1", "ENSMUSG00000084783.1", "ENSMUSG00000031548.7", "ENSMUSG00000057672.16", "ENSMUSG00000002908.17", "ENSMUSG00000031864.16", "ENSMUSG00000089776.1", "ENSMUSG00000109753.1", "ENSMUSG00000031497.9", "ENSMUSG00000004626.14", "ENSMUSG00000110476.1", "ENSMUSG00000055368.14", "ENSMUSG00000031951.8", "ENSMUSG00000053979.4", "ENSMUSG00000031491.10", "ENSMUSG00000109868.1", "ENSMUSG00000087300.1", "ENSMUSG00000062181.14", "ENSMUSG00000088969.1", "ENSMUSG00000015027.11", "ENSMUSG00000031553.16", "ENSMUSG00000086500.1", "ENSMUSG00000031682.5", "ENSMUSG00000055994.15", "ENSMUSG00000031478.16", "ENSMUSG00000031667.16", "ENSMUSG00000110381.1", "ENSMUSG00000031628.9", "ENSMUSG00000084945.1", "ENSMUSG00000031712.10", "ENSMUSG00000110667.1"]
slop = 250000
n_pc = 10
resolution = 1


In [5]:
output_dir = pathlib.Path(chrom)
output_dir.mkdir(exist_ok=True)

## Cell Meta

In [6]:
cell_tidy_data = pd.read_msgpack(
    '/home/hanliu/project/mouse_rostral_brain/study/ClusteringSummary/Summary/TotalClusteringResults.msg'
)
use_clusters = [
    i.replace(' ', '_') for i in cell_tidy_data[cell_tidy_data['CellClass'].isin(['Exc', 'Inh'])]
    ['SubType'].unique() if 'Outlier' not in i
]
len(use_clusters)

145

## ATAC peaks

In [7]:
atac_peak = pd.read_msgpack('/home/hanliu/project/mouse_rostral_brain/study/DMRCluster/SubType.ATAC_peak_merged.msg')
atac_peak = atac_peak.loc[atac_peak.index.map(lambda i: i.startswith(f'Sub{chrom}_'))].copy()

## Gene Info

In [8]:
gene_meta = pd.read_csv(
    '/home/hanliu/ref/mouse/gencode/vm22/gencode.vM22.annotation.gene.flat.tsv.gz',
    index_col='gene_id', sep='\t'
)
gene_meta = gene_meta[gene_meta['chrom'] == chrom].copy()

In [9]:
exon_bed = pd.read_csv('/home/hanliu/ref/mouse/gencode/vm22/genome_anno/exon.all.bed',
                       header=None, sep='\t')
exon_bed.columns = ['chrom', 'start', 'end', 'gene_id', 'gene_name']

## DMR Info

In [10]:
with pd.HDFStore('/home/hanliu/project/mouse_rostral_brain/DMR/SubType/Total/DMRInfo.h5', 'r') as hdf:
    dmr_rate = hdf['Rate']
dmr_rate = dmr_rate.loc[dmr_rate.index.map(lambda i: i.startswith(f'Sub{chrom}_'))].copy()

dmr_corr = pd.read_msgpack(
    '/home/hanliu/project/mouse_rostral_brain/study/DMRGeneCorr/TotalGeneDMRCorrLoop.0.3.msg'
)
dmr_corr = dmr_corr.set_index(['DMR', 'Gene'])
dmr_corr = dmr_corr.loc[dmr_corr.index.get_level_values('DMR').isin(dmr_rate.index)].copy()

dmr_bed = pd.read_csv('/home/hanliu/project/mouse_rostral_brain/DMR/SubType/Total/TotalDMR.nofilter.bed',
                      sep='\t', header=None, index_col=3)
dmr_bed.columns = ['chrom', 'start', 'end']
dmr_bed = dmr_bed[dmr_bed['chrom'] == chrom].copy()

dmr_hits = anndata.read_h5ad('/home/hanliu/project/mouse_rostral_brain/DMR/SubType/Total/TotalHits.HypoDMR.h5ad')
dmr_hits = dmr_hits[dmr_rate.index, :].copy()
dmr_hits = dmr_hits[:, use_clusters].copy()

dmr_annot = anndata.read_h5ad('/home/hanliu/project/mouse_rostral_brain/DMR/SubType/Total/DMRAnnotation.h5ad')
dmr_annot = dmr_annot[dmr_rate.index, :].copy()

In [11]:
dmr_hits.shape[0]

248520

In [12]:
def get_gene(gene_id):
    _gene = gene_meta.loc[gene_id]
    return _gene.name, _gene['chrom'], _gene['start'], _gene['end'], _gene['strand']

## Gene's DMR clustering

In [13]:
def calculate_gene(gene_id):
    # get gene information
    gene_id, _, gene_start, gene_end, _ = get_gene(gene_id)

    # select related DMRs
    related_dmr = dmr_bed[(dmr_bed['start'] > gene_start - slop) &
                          (dmr_bed['end'] < gene_end + slop)].copy()
    
    related_dmr_rate = dmr_rate.loc[related_dmr.index, use_clusters].copy()
    related_dmr_rate.fillna(related_dmr_rate.mean(), axis=0, inplace=True)
    
    # construct Adata
    adata = anndata.AnnData(X=related_dmr_rate.values.copy(),
                            obs=pd.DataFrame([], related_dmr_rate.index),
                            var=pd.DataFrame([], related_dmr_rate.columns))
    sc.pp.scale(adata)
    sc.pp.pca(adata)
    
    pcs = adata.obsm['X_pca'][:, :n_pc]
    sc.pp.neighbors(adata, n_neighbors=int(round(np.log2(adata.shape[0]))), n_pcs=n_pc)
    sc.tl.leiden(adata, resolution=resolution)
    return adata.obs


def get_annotation(gene_id):
    gene_id, _, gene_start, gene_end, strand = get_gene(gene_id)
    tss = gene_start if strand == '+' else gene_end
    gene_cluster = calculate_gene(gene_id)
    this_corr = dmr_corr[dmr_corr.index.get_level_values('Gene') == gene_id]['Corr']
    this_corr.index = this_corr.index.droplevel('Gene')
    gene_cluster['Corr'] = gene_cluster.index.map(this_corr).fillna(0)
    
    this_dmr_bed = dmr_bed.loc[gene_cluster.index]
    dmr_center = (this_dmr_bed['end'] + this_dmr_bed['start']) / 2
    gene_length = gene_end - gene_start
    if strand == '+':
        gene_cluster['reldist_tss'] = (dmr_center - gene_start) / gene_length
    else:
        gene_cluster['reldist_tss'] = (gene_end - dmr_center) / gene_length
    gene_cluster['in_gene_body'] = (gene_cluster['reldist_tss'] > 0) & (gene_cluster['reldist_tss'] < 1)
    
    this_annot = dmr_annot[gene_cluster.index]
    annot_df = pd.DataFrame(this_annot.X.todense(), 
                 index=this_annot.obs_names, columns=this_annot.var_names)
    
    # annotate TE cols
    dna_te = annot_df.columns[20:33]
    gene_cluster['is_dna_te'] = annot_df[dna_te].sum(axis=1) != 0
    
    line_te = annot_df.columns[33:39]
    gene_cluster['is_line_te'] = annot_df[line_te].sum(axis=1) != 0
    
    ltr_te = annot_df.columns[39:45]
    gene_cluster['is_ltr_te'] = annot_df[ltr_te].sum(axis=1) != 0
    
    sine_te = annot_df.columns[45:52]
    gene_cluster['is_sine_te'] = annot_df[sine_te].sum(axis=1) != 0
    
    # this dmr within GOI's gene feature
    gene_cluster['in_intron'] = annot_df['intron'].astype(bool) & gene_cluster['in_gene_body']
    gene_cluster['in_exon'] = annot_df['exon'].astype(bool) & gene_cluster['in_gene_body']
    gene_cluster['in_utr3'] = annot_df['UTR3'].astype(bool) & gene_cluster['in_gene_body']
    gene_cluster['in_utr5'] = annot_df['UTR5'].astype(bool) & gene_cluster['in_gene_body']
    
    # previous mC study
    gene_cluster['feDMR'] = annot_df['feDMR'].astype(bool)
    gene_cluster['adultDMR'] = annot_df['adultDMR'].astype(bool)
    
    
    other_profiles = []
    # DMR hypo call in each cluster
    this_hypo_hits = dmr_hits[gene_cluster.index]
    hits_df = pd.DataFrame(this_hypo_hits.X.todense(), 
                 index=this_hypo_hits.obs_names, columns=this_hypo_hits.var_names)
    hits_df.columns = hits_df.columns.map(lambda i: f'HypoDMR.{i}')
    other_profiles.append(hits_df)
    
    # DMR rate
    related_dmr_rate = dmr_rate.loc[gene_cluster['leiden'].sort_values().index, use_clusters].copy()
    related_dmr_rate.fillna(related_dmr_rate.mean(), axis=0, inplace=True)
    related_dmr_rate.columns = related_dmr_rate.columns.map(lambda i: f'DMRRate.{i}')
    other_profiles.append(related_dmr_rate)
    
    # atac peak
    atac_peak_df = atac_peak.loc[related_dmr_rate.index, use_clusters].copy()
    other_profiles.append(atac_peak_df)
    atac_peak_df.columns = atac_peak_df.columns.map(lambda i: f'ATACPeak.{i}')
    
    dmr_annotation = pd.concat([gene_cluster] + other_profiles, axis=1, sort=True)
    return dmr_annotation

In [15]:
for gene in genes:
    print(gene)
    check_path = output_dir / f'{gene}.DMR_cluster.msg'
    if check_path.exists():
        continue
    
    dmr_annotation = get_annotation(gene)
    dmr_annotation.to_msgpack(output_dir / f'{gene}.DMR_detail.msg', compress='zlib')
    
    cluster_annotation = dmr_annotation.groupby('leiden').mean()
    cluster_annotation.to_msgpack(output_dir / f'{gene}.DMR_cluster.msg', compress='zlib')

ENSMUSG00000025812.17
ENSMUSG00000025810.9
ENSMUSG00000079157.4
ENSMUSG00000063626.6
ENSMUSG00000010936.11
ENSMUSG00000031618.13
ENSMUSG00000001911.16
ENSMUSG00000041949.6
ENSMUSG00000037492.16
ENSMUSG00000046844.6
ENSMUSG00000043668.13
ENSMUSG00000031822.20
ENSMUSG00000031642.9
ENSMUSG00000060924.15
ENSMUSG00000045636.16
ENSMUSG00000039057.11
ENSMUSG00000031930.11
ENSMUSG00000036306.13
ENSMUSG00000036356.15
ENSMUSG00000041308.6
ENSMUSG00000031565.18
ENSMUSG00000062991.8
ENSMUSG00000001995.9
ENSMUSG00000031709.15
ENSMUSG00000031561.16
ENSMUSG00000036469.16
ENSMUSG00000057060.8
ENSMUSG00000089704.9
ENSMUSG00000071064.14
ENSMUSG00000031523.16
ENSMUSG00000031841.19
ENSMUSG00000047747.10
ENSMUSG00000037148.8
ENSMUSG00000031681.16
ENSMUSG00000036510.17
ENSMUSG00000031608.13
ENSMUSG00000031453.16
ENSMUSG00000000759.14
ENSMUSG00000053626.5
ENSMUSG00000031772.17
ENSMUSG00000038291.16
ENSMUSG00000031824.15
ENSMUSG00000031673.6
ENSMUSG00000070056.6
ENSMUSG00000031626.17
ENSMUSG00000001300.16
ENS

ENSMUSG00000057672.16
ENSMUSG00000002908.17
ENSMUSG00000031864.16
ENSMUSG00000089776.1
ENSMUSG00000109753.1
ENSMUSG00000031497.9
ENSMUSG00000004626.14
ENSMUSG00000110476.1
ENSMUSG00000055368.14
ENSMUSG00000031951.8
ENSMUSG00000053979.4
ENSMUSG00000031491.10
ENSMUSG00000109868.1
ENSMUSG00000087300.1
ENSMUSG00000062181.14
ENSMUSG00000088969.1
ENSMUSG00000015027.11
ENSMUSG00000031553.16
ENSMUSG00000086500.1
ENSMUSG00000031682.5
ENSMUSG00000055994.15
ENSMUSG00000031478.16
ENSMUSG00000031667.16
ENSMUSG00000110381.1
ENSMUSG00000031628.9
ENSMUSG00000084945.1
ENSMUSG00000031712.10
ENSMUSG00000110667.1
